# Import lib

In [2]:
from transformers import MBartForCausalLM
import torch
import transformers
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset, DatasetDict
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import BartForConditionalGeneration, BartTokenizer

2024-06-07 09:10:47.540404: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-07 09:10:47.540495: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-07 09:10:47.662565: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Load data & model

In [4]:
model_id = "vinai/bartpho-syllable"
bartpho = AutoModelForSeq2SeqLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

datasets = load_dataset("csv", data_files='/kaggle/input/data-vilexnorm/train.csv')

config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

dict.txt:   0%|          | 0.00/360k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
train_dataset = datasets.remove_columns(['Unnamed: 0'])
train_dataset


DatasetDict({
    train: Dataset({
        features: ['original', 'normalized'],
        num_rows: 8372
    })
})

In [6]:
train_dataset['train'][10]["original"], train_dataset['train'][10]["normalized"]

('Thế nào đc 9.5 thật đó đỉnh vãi ò, đã v mk điểm cao nhất lớp chứ đc cô thưởng 300k',
 'Thế nào đc 9.5 thật đó đỉnh vãi ò, đã vậy mình điểm cao nhất lớp chứ được cô thưởng 300k')

# Feat ex

In [9]:
def preprocess_function(examples):
    max_input_length = 128
    max_target_length = 128
    inputs = [doc for doc in examples["original"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["normalized"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset['train'].column_names)
tokenized_datasets

Map:   0%|          | 0/8372 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8372
    })
})

# Training

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=bartpho)

args = Seq2SeqTrainingArguments(
    "/kaggle/working/",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=40,
    save_steps=2000,
    fp16=True
    )

trainer = Seq2SeqTrainer(
    bartpho,
    args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

In [ ]:
trainer.save_model("/kaggle/working/model_final")